<a href="https://colab.research.google.com/github/saykim/ds/blob/main/%EC%9D%B4%EB%AF%B8%EC%A7%80%EB%B6%84%EB%A5%98%EB%B9%84%EA%B5%90_fastai_keras_230221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---
* 각 모델의 성능 비교를 위해서는 각각의 모델을 학습하고 검증하는 과정이 필요합니다. 그래서 일단 두 모델을 간단히 소개하고, 그다음에 해당 모델들을 학습하고 평가하는 과정을 거쳐서 성능을 비교해보도록 하겠습니다.

* 먼저, Fastai는 PyTorch를 기반으로 한 딥러닝 라이브러리로, 간단한 코드 작성으로 고수준의 인공지능 모델을 학습할 수 있도록 지원합니다. Fastai는 사용자 친화적인 API와 다양한 이미지 처리 기능을 제공하여 딥러닝 모델 개발에 큰 도움이 됩니다.

* 반면에, 위에서 작성한 코드는 Keras를 이용하여 간단한 CNN 모델을 구성하여 이미지 분류 문제를 해결한 것입니다. Keras는 TensorFlow를 기반으로 한 딥러닝 라이브러리로, 사용자 친화적인 API를 제공하여 쉽게 딥러닝 모델을 구성할 수 있습니다.

* Fastai와 Keras 모두 이미지 분류 문제를 해결하는 데에 적합한 라이브러리이지만, 성능 측면에서는 데이터셋의 특성에 따라 다를 수 있습니다. 따라서 고양이와 개 분류 문제를 해결할 때, 두 라이브러리 모두 좋은 성능을 보일 것으로 예상됩니다.

* 하지만, 엑스레이 이미지 분류 문제에서는 성능에 차이가 발생할 수 있습니다. 이는 엑스레이 이미지 분류 문제가 고양이와 개 분류 문제와는 데이터의 특성이 달라서 발생할 수 있는 현상입니다. 엑스레이 이미지는 고해상도의 이미지가 많고, 이미지 내부에 다양한 패턴과 텍스처가 존재하기 때문에 고차원적인 특징을 추출하고 분류하는 능력이 필요합니다. 따라서, 이러한 고차원적인 특징 추출 능력이 좋은 모델이 더 나은 성능을 보일 것으로 예상됩니다.

* 이제 각 라이브러리에서 고양이와 개 이미지 분류 문제와 엑스레이 이미지 분류 문제를 해결하는 모델을 학습하고 평가하여 성능을 비교해보도록 하겠습니다.


# Fastai

In [ ]:
from fastai.vision.all import *

path = untar_data(URLs.PETS)/'images'

def is_cat(x):
    return x[0].isupper()

dls = ImageDataLoaders.from_name_func(path, get_image_files(path), valid_pct=0.2, seed=42, 
                                      label_func=is_cat, item_tfms=Resize(460), batch_tfms=aug_transforms(size=224, min_scale=0.75))

learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(1)


> 위 코드는 Fastai를 이용하여 고양이와 개 이미지 분류 문제를 해결하는 모델을 학습하고 평가한 결과입니다. 데이터셋으로는 fastai 라이브러리에서 제공하는 PETS 데이터셋을 이용하였습니다. is_cat 함수를 이용하여 레이블을 지정하고, ImageDataLoaders를 이용하여 데이터셋을 불러왔습니다. 모델은 resnet34를 이용하였으며, fine_tune 메서드를 이용하여 모델을 학습하였습니다.

# Keras

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping

img_width, img_height = 150, 150
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='binary')

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    class_weight=class_weight,
    callbacks=[early_stopping]
)


> 위 코드에서는 EarlyStopping 콜백 함수를 이용하여 val_loss 값이 3번 이상 개선되지 않으면 학습을 중단합니다. fit_generator 함수를 이용하여 학습을 수행하며, class_weight를 이용하여 클래스 불균형 문제를 해결하였습니다.



## 모델간 비교

In [ ]:
# 고양이와 개 이미지 분류 문제 평가
loss, acc = learn.validate()
print(f'Fastai model accuracy: {acc*100:.2f}%')

score = model.evaluate_generator(validation_generator, nb_validation_samples // batch_size)
print(f'Keras model accuracy: {score[1]*100:.2f}%')

# 엑스레이 이미지 분류 문제 평가
from tensorflow.keras.preprocessing import image

def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x /= 255.

    pred = model.predict(x)[0][0]
    return 'Positive' if pred > 0.5 else 'Negative'

xray_model = load_model('xray_model.h5')
xray_model.evaluate(x_test, y_test)

# 예측
predict_image(xray_model, 'xray_image.png')


> 위 코드에서는 먼저 고양이와 개 이미지 분류 문제에서 각 모델의 성능을 측정합니다. Fastai 모델은 validate 메서드를 이용하여 성능을 측정하였으며, Keras 모델은 evaluate_generator 함수를 이용하여 성능을 측정하였습니다. 두 모델의 성능은 각각 acc와 score[1] 값으로 확인할 수 있습니다.

> 또한, 엑스레이 이미지 분류 문제에서는 미리 학습된 모델을 불러와서 성능을 평가하였습니다. load_model 함수를 이용하여 불러온 모델은 evaluate 메서드를 이용하여 성능을 측정하였습니다. 이후, predict_image 함수를 이용하여 샘플 이미지의 분류 결과를 예측하였습니다.



이렇게 성능을 측정하여 두 모델의 성능을 비교할 수 있습니다.